In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns = 50

In [3]:
def read_in_csv(file_path='./parking-geo.csv'):
    # let's be memory efficient when loading our data
    dtypes_dict = \
    {
        'ticket_number': np.int32,
        'violation_location': str,
        'license_plate_number': str,
        'license_plate_state': 'category',
        'license_plate_type': 'category',
        'zipcode': str,
        'violation_code': 'category',
        'violation_description': 'category',
        'unit': 'category',
        'unit_description': 'category',
        'vehicle_make': 'category',
        'fine_level1_amount': np.int32,
        'fine_level2_amount': np.int32,
        'current_amount_due': np.float64,
        'total_payments': np.float64,
        'ticket_queue': 'category',
        'notice_level': 'category',
        'hearing_disposition': 'category',
        'notice_number': np.int32,
        'dismissal_reason': str,
        'officer': str,
        'address': str,
        'license_hash': str,
        'year': np.int32,
        'month': 'category',
        'hour': 'category',
        'penalty': np.float64,
        'ward': 'category',
        'geocode_accuracy': np.float64,
        'geocode_accuracy_type': 'category',
        'geocoded_address': str,
        'geocoded_lng': str,
        'geocoded_lat': str,
        'geocoded_city': 'category',
        'geocoded_state': 'category'
    }
    #still better than strings
    parse_dates_list = \
    [
        'issue_date',
        'ticket_queue_date',
    ]
    
    # read csv into memory -- this takes quite a while
    df = pd.read_csv(file_path, dtype=dtypes_dict, parse_dates=parse_dates_list)
    return df

In [4]:
def calculate_summary_stats(df_raw, min_year = 1995, max_year = 2019):
    
    # private helper function
    def rank_series(series):
        out_series = series.rank(ascending=False)
        return out_series
    
    df_filtered = df_raw[
        (df_raw['year'] > min_year) & (df_raw['year'] < max_year) & 
        (df_raw['geocode_accuracy_type'].isin(['rooftop', 'range_interpolation', 'intersection', 'point'])) & 
        (df_raw['geocoded_city'] == 'Chicago')
    ]
    df_filtered_na = df_filtered[df_filtered['ward'].isnull()]
    df_filtered = df_filtered[df_filtered['ward'].notnull()]

    # calculate base dataframes
    df_dict = dict()
    df_dict['filtered'] = df_filtered
    df_dict['police_tickets'] = df_filtered[
        df_filtered['unit_description'].isin(['CPD', 'CPD-Other', 'CPD-Airport'])
    ]
    df_dict['contested_tickets'] = df_filtered[
        df_filtered['hearing_disposition'].isin(['Liable', 'Not Liable'])
    ]
    df_dict['paid_tickets'] = df_filtered[
        df_filtered['ticket_queue'] == 'Paid'
    ]
    df_dict['dismissed_tickets'] = df_filtered[
        df_filtered['ticket_queue'] == 'Dismissed'
    ]
    df_dict['seizedorsuspended_tickets'] = df_filtered[
        df_filtered['notice_level'].isin(['SEIZ', 'DLS'])
    ]
    df_dict['bankruptcy_tickets'] = df_filtered[
        df_filtered['ticket_queue'] == 'Bankruptcy'
    ]

    # group dataframes by ward
    gb_dict = dict()
    for key in df_dict:
        gb_dict[key] = df_dict[key].groupby('ward')

    # calculate the different stats
    out_dict = dict()
    ticket_count = gb_dict['filtered']['ticket_number'].count()
    out_dict['ticket_count'] = ticket_count
    out_dict['current_amount_due'] = gb_dict['filtered']['current_amount_due'].sum()
    out_dict['fine_level1_amount'] = gb_dict['filtered']['fine_level1_amount'].sum()
    out_dict['total_payments'] = gb_dict['filtered']['total_payments'].sum()
    out_dict['avg_per_ticket'] = out_dict['fine_level1_amount']/ticket_count
    out_dict['paid_pct'] = out_dict['total_payments']/(out_dict['current_amount_due']+out_dict['total_payments'])
    out_dict['police_ticket_count'] = gb_dict['police_tickets']['ticket_number'].count()
    out_dict['police_ticket_count_pct'] = out_dict['police_ticket_count'] / ticket_count
    out_dict['contested_ticket_count'] = gb_dict['contested_tickets']['ticket_number'].count()
    out_dict['contested_ticket_count_pct'] = out_dict['contested_ticket_count'] / ticket_count
    out_dict['paid_ticket_count'] = gb_dict['paid_tickets']['ticket_number'].count()
    out_dict['paid_ticket_count_pct'] = out_dict['paid_ticket_count'] / ticket_count
    out_dict['dismissed_ticket_count'] = gb_dict['dismissed_tickets']['ticket_number'].count()
    out_dict['dismissed_ticket_count_pct'] = out_dict['dismissed_ticket_count'] / ticket_count
    out_dict['seizedorsuspended_ticket_count'] = gb_dict['seizedorsuspended_tickets']['ticket_number'].count()
    out_dict['seizedorsuspended_ticket_count_pct'] = out_dict['seizedorsuspended_ticket_count'] / ticket_count
    out_dict['bankruptcy_ticket_count'] = gb_dict['bankruptcy_tickets']['ticket_number'].count()
    out_dict['bankruptcy_ticket_count_pct'] = out_dict['bankruptcy_ticket_count'] / ticket_count

    # calculate ranks; combine and format output dataframe
    df_out = pd.DataFrame()
    for key in out_dict:
        df_out[key] = out_dict[key]
        df_out[key+'_rank'] = rank_series(out_dict[key]).astype(int)
    df_out.index = df_out.index.astype(int)
    df_out = df_out.sort_index()
    
    return df_out

In [5]:
%%time
# takes quite a while...
df = read_in_csv()

CPU times: user 9min 25s, sys: 2min 29s, total: 11min 54s
Wall time: 12min 14s


In [6]:
%%time
df_1996to2018 = calculate_summary_stats(df, min_year=1995, max_year=2019)

CPU times: user 1min 19s, sys: 2min 18s, total: 3min 37s
Wall time: 4min 7s


In [7]:
%%time
df_2013to2017 = calculate_summary_stats(df, min_year=2012, max_year=2018)

CPU times: user 18.6 s, sys: 9.11 s, total: 27.7 s
Wall time: 21.5 s


In [8]:
df_1996to2018

ticket_count  ticket_count_rank  current_amount_due  \
ward                                                        
1          1720717                  8        3.940247e+07   
2          2081261                  3        3.431393e+07   
3          1189111                 12        4.315614e+07   
4          1783330                  6        4.619477e+07   
5          1082520                 16        3.955258e+07   
6           842531                 23        4.985787e+07   
7           605630                 37        3.619122e+07   
8           644121                 36        3.458872e+07   
9           426441                 43        2.456196e+07   
10          378418                 45        1.751554e+07   
11          691235                 34        1.856930e+07   
12          829417                 25        3.420937e+07   
13          315842                 47        1.140697e+07   
14          654187                 35        2.733252e+07   
15          792182                 29        3.814869e+07   
16          819022                 26        5.112076e+07   
17          716216                 33        4.438406e+07   
18          254129                 48        1.253572e+07   
19          226082                 49        7.715441e+06   
20          807223                 28        5.046665e+07   
21          543507                 39        3.266183e+07   
22          876420                 21        3.958049e+07   
23          586017                 38        1.817335e+07   
24          850825                 22        5.401623e+07   
25         1721025                  7        4.275620e+07   
26         1026314                 20        3.887982e+07   
27         2018335                  5        6.399331e+07   
28         1354565                  9        7.065656e+07   
29          756302                 30        4.436703e+07   
30          724676                 32        2.480514e+07   
31          816381                 27        2.892891e+07   
32         1099169                 15        2.020482e+07   
33         1050654                 17        3.217766e+07   
34          424789                 44        2.886192e+07   
35         1141752                 14        3.752917e+07   
36          473826                 42        1.855162e+07   
37          734715                 31        4.703888e+07   
38          338755                 46        9.475390e+06   
39          489431                 41        1.273742e+07   
40         1035475                 19        2.473925e+07   
41          190452                 50        4.843758e+06   
42         5077427                  1        1.019662e+08   
43         2070786                  4        2.618685e+07   
44         2557809                  2        3.516384e+07   
45          506116                 40        1.331577e+07   
46         1229482                 11        2.827049e+07   
47         1285899                 10        2.069428e+07   
48         1037712                 18        2.547922e+07   
49         1155988                 13        3.829323e+07   
50          834795                 24        2.206733e+07   

      current_amount_due_rank  fine_level1_amount  fine_level1_amount_rank  \
ward                                                                         
1                          16           100835215                        8   
2                          24           117298610                        4   
3                          12            73217160                       10   
4                           9           101895980                        7   
5                          15            68784395                       13   
6                           7            60035045                       19   
7                          21            42136265                       37   
8                          23            44527970                       34   
9                          36            30701615    

In [9]:
df_2013to2017

ticket_count  ticket_count_rank  current_amount_due  \
ward                                                        
1           423120                  5        8.875342e+06   
2           455095                  3        8.484861e+06   
3           302444                 10        1.467788e+07   
4           318959                  8        1.222940e+07   
5           252692                 12        1.488461e+07   
6           203418                 21        2.127402e+07   
7           142450                 39        1.529488e+07   
8           152634                 36        1.504893e+07   
9           107925                 42        1.087846e+07   
10           86795                 45        6.307718e+06   
11          185986                 22        5.526755e+06   
12          168495                 32        9.062659e+06   
13           83126                 46        4.005094e+06   
14          163484                 34        8.711852e+06   
15          176710                 30        1.191936e+07   
16          180960                 26        1.932971e+07   
17          174634                 31        1.905989e+07   
18           67538                 48        5.366925e+06   
19           54037                 49        3.041398e+06   
20          215139                 16        2.203540e+07   
21          146967                 38        1.549427e+07   
22          178028                 28        1.006936e+07   
23          147537                 37        5.661258e+06   
24          210131                 18        2.310028e+07   
25          442655                  4        1.106290e+07   
26          226812                 14        9.987975e+06   
27          417998                  6        1.963656e+07   
28          316065                  9        2.888483e+07   
29          178039                 27        1.803229e+07   
30          165844                 33        6.473144e+06   
31          183242                 25        7.876283e+06   
32          226608                 15        3.888104e+06   
33          183762                 23        5.262404e+06   
34          117784                 40        1.400708e+07   
35          209988                 19        7.263938e+06   
36          110165                 41        4.954108e+06   
37          183310                 24        1.927644e+07   
38           69827                 47        2.476610e+06   
39          103665                 43        3.414797e+06   
40          211875                 17        4.676262e+06   
41           46107                 50        1.434325e+06   
42         1097695                  1        2.769707e+07   
43          380718                  7        5.326003e+06   
44          520445                  2        7.965664e+06   
45          102532                 44        3.190052e+06   
46          242468                 13        5.787010e+06   
47          260425                 11        3.589766e+06   
48          161715                 35        3.843112e+06   
49          204934                 20        7.079901e+06   
50          177127                 29        5.262148e+06   

      current_amount_due_rank  fine_level1_amount  fine_level1_amount_rank  \
ward                                                                         
1                          24            30520260                        6   
2                          26            33039930                        4   
3                          15            23925780                       10   
4                          17            25149195                        9   
5                          14            21604550                       11   
6                           5            19595595                       14   
7                          12            13945980                       35   
8                          13            14817615                       30   
9                          20            10442210    

In [10]:
df_1996to2018.to_csv('df_1996to2018.csv')
df_1996to2018.to_csv()

'ward,ticket_count,ticket_count_rank,current_amount_due,current_amount_due_rank,fine_level1_amount,fine_level1_amount_rank,total_payments,total_payments_rank,avg_per_ticket,avg_per_ticket_rank,paid_pct,paid_pct_rank,police_ticket_count,police_ticket_count_rank,police_ticket_count_pct,police_ticket_count_pct_rank,contested_ticket_count,contested_ticket_count_rank,contested_ticket_count_pct,contested_ticket_count_pct_rank,paid_ticket_count,paid_ticket_count_rank,paid_ticket_count_pct,paid_ticket_count_pct_rank,dismissed_ticket_count,dismissed_ticket_count_rank,dismissed_ticket_count_pct,dismissed_ticket_count_pct_rank,seizedorsuspended_ticket_count,seizedorsuspended_ticket_count_rank,seizedorsuspended_ticket_count_pct,seizedorsuspended_ticket_count_pct_rank,bankruptcy_ticket_count,bankruptcy_ticket_count_rank,bankruptcy_ticket_count_pct,bankruptcy_ticket_count_pct_rank\n1,1720717,8,39402474.29997046,16,100835215,8,92243562.2200177,6,58.600696686323204,38,0.7006938048302891,7,598783,14,0.

In [11]:
df_2013to2017.to_csv('df_2013to2017.csv')
df_2013to2017.to_csv()

'ward,ticket_count,ticket_count_rank,current_amount_due,current_amount_due_rank,fine_level1_amount,fine_level1_amount_rank,total_payments,total_payments_rank,avg_per_ticket,avg_per_ticket_rank,paid_pct,paid_pct_rank,police_ticket_count,police_ticket_count_rank,police_ticket_count_pct,police_ticket_count_pct_rank,contested_ticket_count,contested_ticket_count_rank,contested_ticket_count_pct,contested_ticket_count_pct_rank,paid_ticket_count,paid_ticket_count_rank,paid_ticket_count_pct,paid_ticket_count_pct_rank,dismissed_ticket_count,dismissed_ticket_count_rank,dismissed_ticket_count_pct,dismissed_ticket_count_pct_rank,seizedorsuspended_ticket_count,seizedorsuspended_ticket_count_rank,seizedorsuspended_ticket_count_pct,seizedorsuspended_ticket_count_pct_rank,bankruptcy_ticket_count,bankruptcy_ticket_count_rank,bankruptcy_ticket_count_pct,bankruptcy_ticket_count_pct_rank\n1,423120,5,8875342.4300019,24,30520260,6,28610635.009996008,5,72.13145207033466,48,0.7632356674116806,6,74299,28,0.1755